In [ ]:
# Delft 372 parameters 
# added Current velocity (breaks at >=0.48 m/s - do not exceed), Wind as force from velocity and direction
# MPC ver 2.6  - Final version for submission


# ----------------------------------------------------------------------------------
# 1) IMPORTS
# ----------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import math 
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch, Circle
from scipy.interpolate import interp1d, RegularGridInterpolator
# (Optional) from scipy.interpolate import UnivariateSpline
# import other libraries if needed
from matplotlib.patches import FancyArrowPatch
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import time


In [ ]:
# 2) SIMULATION CONDITIONS

#     This dictionary holds simulation parameters, environment settings,
#     initial conditions, and placeholders for controller gains, etc.
# ----------------------------------------------------------------------------------
sim_params = {
    # Time settings
    "dt": 0.1,        # time step
    "t_max": 600.0,   # total simulation time (seconds)
    "initial_state": np.array([0.5, 0.5, -np.pi/3, 0.0, 0.0, 0.0]),  # [x, y, psi, u, v, r]

    # Current parameters
    "current_velocity": 0.3,  #0.34,  # m/s # 0.32 m/s is the max value before the ship starts to break
    "current_dir_deg": 0,  # degrees from global x-axis
    
    # Wind parameters
    "wind_speed" : 14, #m/s
    "wind_dir_deg": 0,       # degrees from global x-axis
   




    # Dynamics parameters
    "gust_period": 90,       # seconds (occurance)
    "gust_duration": 0,      # seconds
    "gust_speed_increase": 5,         # m/s
    "gust_angle_increase" : 20,       # degrees
    


    "rho_air": 1.225,  # kg/m^3
    


    "anchor_a" : 0.2,   # example coefficient
    "anchor_b" : 3,     # example exponent
    "anchor_r0" : 4,     # shift: no force for r <= 3.5 m

    # Wave parameters
    "wave_Hs": 0.5,       # significant wave height
    "wave_Tp": 6.75,      # peak period
    "wave_depth": 5.0,    # water depth
    "g": 9.81,

    # For wave discretization
    "N_freq": 50,
    "omega_min": 0.1,
    # "omega_max" about 4*(2*pi/Tp):
    # We'll compute it below.


    # Controller placeholders
    "controller_surge": 0.0,  # small constant surge force (N)
    "controller_sway": 0.0,
    "controller_yaw":  0.0,

    # Wind parameters
    "c1" : 0.2, #time phases
    "c2" : 0.4, 
    "c3" : 0.6,
    "c4" : 0.8,

 
    # # Anchor force interpolation data
    # "anchor_r_points": np.array([0,   2,   3,   4,   5, 6, 7,  8]),
    # "anchor_F_points": np.array([0, 0.1, 0.2, 0.5, 1.2, 3, 15, 40]),
    
}

# Additional computations or initial setups:
sim_params["omega_max"] = 4.0 * (2.0 * np.pi / sim_params["wave_Tp"])

# # Anchor force interpolation
# anchor_force_interp = interp1d(
#     sim_params["anchor_r_points"], 
#     sim_params["anchor_F_points"], 
#     kind='linear', 
#     fill_value="extrapolate"
# )
# sim_params["anchor_force_interp"] = anchor_force_interp


ship_params = {

    # Ship physical parameters (mass, inertia, etc.)
    "m": 87.07 ,        # mass
    "Iz": 58.9,       # moment of inertia
    "xg": -0.09,      # CoG x-location
    "Lpp": 3,     # length between perpendiculars # OMG SHOULD HAVE BEEN 3!!!
    "A_lat":  1.5,    # lateral area of the ship m2
    "A_front": 0.5,   # frontal area of the ship m2
    "Cd_wind": 0.2,  # drag coefficient for wind
    "x_cp": 0,     # center of pressure location forward of CG
    "y_cp" : 0,      # center of pressure location to the side of CG

    # Added mass
    "Xudot": -9.2,    # Assumed value  
    "Yvdot": -92,
    "Yrdot": 1.26,
    "Nvdot": 29.5,
    "Nrdot": -184,

    # Linear damping
    "Xu": -28.65,       #Assumed value 
    
    # "Yv": -57.3,
    "Yv": -157.3,
    
    "Yr": -38.2,
    # "Nv": -38.6,
    
    "Nv": 0,
    "Nr": -27.3,

    # Nonlinear damping
    "Xuu": -4.89,
    "Xvr": 2.26,

    "Yvr": -361,
    "Yvvv": -1347,
    "Yvvvvv": 13027,
    
    "Yur" :-441,
    # "Yur" :-44.1,
    
    "Yurr": 7344,
    # "Yurr": 734.4,

    "Nrr": -504,
    "Nvvv": 249,
    "Nvvr": 517,

    "Nuv": - 0,
    # "Nuv": - 113,

    "Nuvr": 1405,
    
    "Nuvvv": -2405,
    # "Nuvvv": -240,

    #     # RAO CSV data (paths to files, if needed)
    # "amp_surge_path":   "Surge_Force_RAO.csv",
    # "phase_surge_path": "Surge_ForcePhase_RAO.csv",
    # "amp_sway_path":    "Sway_Force_RAO.csv",
    # "phase_sway_path":  "Sway_ForcePhase_RAO.csv",
}


#     # Tuning weights for cost
# Q_nu = np.diag([0,50,50])                      # VELOCITY
# Q_nu_terminal = np.diag([0,100,100])

# Q_eta = np.diag([0.01,0.01,10])                 # POSITION
# Q_eta_terminal = np.diag([0.01,0.01,10])
# Q_tau = np.diag([0,0,0])               # PROPULSION



In [ ]:
# 3) SYSTEM DYNAMICS: M, C, D, compute_dstate

# ----------------------------------------------------------------------------------

def compute_mass_matrix(ship_params): 
    # relative current velocity derivative is zero, assuming its not changing in time 
    # then nu_rel_dot = 0 => nu_dot = nu_rel_dot
    """
    Returns the system mass matrix (including added mass).
    """
    m = ship_params["m"]
    Iz = ship_params["Iz"]
    xg = ship_params["xg"]
    Xudot = ship_params["Xudot"]
    Yvdot = ship_params["Yvdot"]
    Yrdot = ship_params["Yrdot"]
    Nvdot = ship_params["Nvdot"]
    Nrdot = ship_params["Nrdot"]

    return np.array([
        [m - Xudot,           0,               0],
        [0,         m - Yvdot,   m*xg - Yrdot ],
        [0,         m*xg - Nvdot,    Iz - Nrdot]
    ])

def compute_coriolis_rb(ship_params, u, v, r):
    """
    C_RB(u,v,r) for the rigid-body part only.
    """
    m  = ship_params["m"]
    xg = ship_params["xg"]

    # Rigid-body Coriolis
    #   C_RB = [[0,  0,   -m(xg*r + v)],
    #           [0,  0,    m*u       ],
    #           [m(xg*r + v), -m*u,  0]]
    return np.array([
        [0,           0,        -m*(xg*r + v)],
        [0,           0,         m*u        ],
        [m*(xg*r + v), -m*u,     0          ]
    ])

def compute_coriolis_added(ship_params, u_r, v_r, r):
    """
    C_A(u_r, v_r, r) for the added-mass part, 
    where (u_r, v_r) are relative velocities.
    """
    Yvdot = ship_params["Yvdot"]
    Yrdot = ship_params["Yrdot"]
    Nvdot = ship_params["Nvdot"]
    Xudot = ship_params["Xudot"]
    # c13 = Yvdot * v_r + 0.5*(Nvdot - Yrdot)*r
    # c23 = -Xudot * u_r

    c13 = Yvdot*v_r + 0.5*(Nvdot + Yrdot)*r
    c23 = -Xudot*u_r

    return np.array([
        [0,      0,    c13],  #check the signs
        [0,      0,    c23],
        [-c13, -c23,   0  ]
    ])

# def compute_coriolis_total(ship_params, u, v, r, u_r, v_r):
#     """
#     Returns C_RB(u,v,r) + C_A(u_r,v_r,r).
#     """
#     C_rb = compute_coriolis_rb(ship_params, u, v, r)
#     C_a  = compute_coriolis_added(ship_params, u_r, v_r, r)
#     return C_rb + C_a

def compute_damping(ship_params, u_r, v_r, r):
    """
    D_tot(u_r, v_r, r) = D_linear + D_nonlinear
    using relative velocities (u_r, v_r).
    """
    # 1) Linear damping
    Xu = ship_params["Xu"]
    Yv = ship_params["Yv"]
    Yr = ship_params["Yr"]
    Nv = ship_params["Nv"] 
    Nr = ship_params["Nr"]

    D_l = np.array([
        [-Xu,   0,     0   ],
        [0,    -Yv,   -Yr ],
        [0,    -Nv,   -Nr ]
    ])

    # 2) Nonlinear damping
    Xuu    = ship_params["Xuu"]
    Xvr    = ship_params["Xvr"]
    Yvr    = ship_params["Yvr"] 
    Yvvv   = ship_params["Yvvv"]
    Yvvvvv = ship_params["Yvvvvv"]  
    Yur    = ship_params["Yur"]
    Yurr   = ship_params["Yurr"]
    Nvvv   = ship_params["Nvvv"]
    Nvvr   = ship_params["Nvvr"]
    Nuv    = ship_params["Nuv"] 
    Nuvr   = ship_params["Nuvr"]
    Nuvvv  = ship_params["Nuvvv"]
    Nrr    = ship_params["Nrr"]

    # For relative velocities: (u_r, v_r)
    d11 = Xuu * u_r
    d13 = Xvr * v_r
    d22 = Yvvv*v_r**2 + Yvvvvv*v_r**4 + Yvr*abs(r)
    d23 = Yur*u_r + Yurr*u_r*r*abs(r)
    d31 = Nuv*v_r + Nuvvv*v_r**3 + Nuvr*v_r*abs(r) 
    d32 = Nvvv*v_r**2 + Nvvr*v_r*abs(r)
    d33 = Nrr*abs(r)

    D_nl =  np.array([
        [-d11,    0,      -d13  ],
        [0,      -d22,   -d23 ],
        [-d31,   -d32,   -d33 ]
    ])

    return D_l + D_nl

def compute_dstate(ship_params, sim_params, state, tau_ext):
    """
    Computes the time derivative of state [x, y, psi, u, v, r],
    including a STEADY current in global frame.

    sim_params should contain:
        "current_velocity": e.g. 0.5   # m/s
        "current_dir_deg": e.g. 90.0   # from global x-axis
    """
    x, y, psi, u, v, r = state  # unpack

    # 1) Kinematics: body->inertial
    J = np.array([
        [np.cos(psi), -np.sin(psi), 0],
        [np.sin(psi),  np.cos(psi), 0],
        [0,            0,           1]
    ])
    eta_dot = J @ np.array([u, v, r])   # (x_dot, y_dot, psi_dot)

    # 2) Current in global frame
    Uc = sim_params.get("current_velocity", 0.0)    # speed m/s
    beta_deg = sim_params.get("current_dir_deg", 0.0)  # direction from x-axis
    beta_rad = np.deg2rad(beta_deg)

    # Global current components:
    Ucx_g = Uc * np.cos(beta_rad)
    Ucy_g = Uc * np.sin(beta_rad)

    # 3) Transform current to BODY frame
    #    (u_c^b, v_c^b) = R^T(psi) * (Ucx_g, Ucy_g)
    cos_psi = np.cos(psi)
    sin_psi = np.sin(psi)
    # R^T(psi) = [[ cos(psi), sin(psi)],
    #             [-sin(psi), cos(psi)]]
    u_c_b =  Ucx_g*cos_psi + Ucy_g*sin_psi
    v_c_b = -Ucx_g*sin_psi + Ucy_g*cos_psi

    # 4) Relative velocities
    u_r = u - u_c_b
    v_r = v - v_c_b

    # 5) Mass
    M = compute_mass_matrix(ship_params)

    # 6) Coriolis
    #    Rigid-body uses (u, v, r), added mass uses (u_r, v_r, r)
    C_rb = compute_coriolis_rb(ship_params, u, v, r)
    C_a = compute_coriolis_added(ship_params, u_r, v_r, r)

    # 7) Damping with relative velocity
    D = compute_damping(ship_params, u_r, v_r, r)

    # 8) State-space form: M * nu_dot + C nu + D nu_r = tau_ext
    nu   = np.array([u, v, r])
    nu_r = np.array([u_r, v_r, r])  # or r_r if you want a separate variable

    rhs = tau_ext - C_rb @ nu - C_a @ nu_r - D @ nu_r
    nu_dot = np.linalg.solve(M, rhs)

    # 9) Return the concatenated derivative
    return np.concatenate((eta_dot, nu_dot)) , nu_dot

In [ ]:
# 4) ENVIRONMENT & FORCES (Anchor)

# ----------------------------------------------------------------------------------

def rotation_matrix(psi):
    """2D rotation matrix, global->body or vice versa depending on usage."""
    return np.array([
        [ np.cos(psi), -np.sin(psi)],
        [ np.sin(psi),  np.cos(psi)]
    ])



def compute_anchor_force(ship_params, sim_params, x, y, psi):
    """
    Evaluate anchor force in ship frame (and return its global magnitude)
    using a shifted power-law relation:
    
        F(r) = 0                if r <= r0
             = a*(r - r0)^b     if r > r0,
             
    where r is the distance from the anchor (global origin) to the bow.
    
    Parameters
    ----------
    ship_params : dict
        Must include:
          "Lpp" : float    # length between perpendiculars
          "xg"  : float    # x-location of CG (for moment calculation)
    sim_params : dict
        Must include:
          "anchor_a" : float   # power law coefficient
          "anchor_b" : float   # power law exponent
          "anchor_r0": float   # shift distance (e.g., 3.0 m)
    x, y, psi : float
        The ship state (position and heading, with psi in radians).
    
    Returns
    -------
    tau_anch : np.array (3,)
        [Fx, Fy, Mz] in the ship’s body frame.
    mag_global : float
        Global magnitude of the anchor force.
    """
    a = sim_params.get("anchor_a", 0.0025)
    b = sim_params.get("anchor_b", 5.32)
    r0 = sim_params.get("anchor_r0", 3.0)
    
    Lpp = ship_params["Lpp"]
    xg  = ship_params["xg"]

    # Bow location in global coordinates   
    bow_x = x + 0.5 * Lpp * np.cos(psi)
    bow_y = y + 0.5 * Lpp * np.sin(psi)

    # Distance from anchor (0,0) to bow
    dist = np.hypot(bow_x, bow_y)
    if dist < 1e-6:
        dist = 1e-6

    # Compute the force magnitude using the shifted power law.
    if dist <= r0:
        F_mag = 0.0
    else:
        F_mag = a * ((dist - r0) ** b)

    # Direction: force acts from the bow toward the anchor.
    Fx_g = -F_mag * (bow_x / dist)
    Fy_g = -F_mag * (bow_y / dist)

    # Convert to ship body frame
    R = rotation_matrix(psi)  # assuming this returns a 2x2 rotation matrix
    F_ship = R.T @ np.array([Fx_g, Fy_g])  # force in body frame

    # Compute moment about CG (force applied at the bow)
    F_anch_moment = F_ship[1] * (Lpp/2 - xg)
    tau_anch = np.array([F_ship[0], F_ship[1], F_anch_moment])

    return tau_anch, np.hypot(Fx_g, Fy_g)




In [ ]:
# 5) ENVIRONMENT & FORCES (Wind)
# ----------------------------------------------------------------------------------

# constant wind
def compute_wind_force(sim_params, psi):
    """
    For now, a constant wind force in global frame -> transform to body frame.
    """
    F_wind_mag = sim_params["wind_force_mag"]  # consider a time-varying wind force
    wind_dir_deg = sim_params["wind_dir_deg"]  #consider a time-varying wind direction
    wind_dir_rad = np.deg2rad(wind_dir_deg)

    # Global wind
    Fx_g = F_wind_mag * np.cos(wind_dir_rad)
    Fy_g = F_wind_mag * np.sin(wind_dir_rad)

    # Convert to ship frame
    R = rotation_matrix(psi)
    F_wind_ship = R.T @ np.array([Fx_g, Fy_g])  # [Fx_ship, Fy_ship]
    # No moment for now
    return np.array([F_wind_ship[0], F_wind_ship[1], 0.0])



def compute_wind_force_3dof(u, v,
    psi,
    ship_params, 
    sim_params
):
    """
    Compute wind force in BODY frame [Fx, Fy, Mz], 
    for a small catamaran with low freeboard.
    
    Inputs
    ------
    x, y, psi : floats
        Current ship position and heading [rad] in the global frame 
        (x,y are not strictly needed unless you want position-based logic).
    ship_params : dict
        Should include:
            "Loa" : float   # overall length ~3 m
            "x_cp" : float  # x-loc of center of pressure (relative to CG) in body frame
            "y_cp" : float  # y-loc of center of pressure (relative to CG) in body frame
            "A_w" : float   # approx. above-water lateral area (m^2)
            "Cd_wind" : float  # drag coefficient
    sim_params : dict
        Should include:
            "wind_speed" : float       # wind speed (m/s)
            "wind_dir_deg" : float     # direction from global x-axis, CCW positive
            "rho_air" : float          # air density (kg/m^3), ~1.225
    
    Returns
    -------
    wind_body : np.array of shape (3,)
        [Fx, Fy, Mz] in BODY frame
    """

    
    # 1) Extract parameters
    U_wind  = sim_params.get("wind_speed", 0.0)   # m/s
    beta_deg= sim_params.get("wind_dir_deg", 0.0) # from global x-axis
    beta_rad= np.deg2rad(beta_deg)
    rho_air = sim_params.get("rho_air", 1.225)
    
    Cd  = ship_params.get("Cd_wind", 0.2) # assuming it is the same for all directions
    A_l = ship_params.get("A_lat", 1.5)  # e.g. 3 m * 0.5 m freeboard => 1.5 m^2
    A_f = ship_params.get("A_front", 0.5)  # e.g. 0.5 m^2
    Lpp = ship_params["Lpp"]

    u_rw = u - U_wind * np.cos(beta_rad - psi)
    v_rw = v - U_wind * np.sin(beta_rad - psi)
    U_wind_rel = np.sqrt(u_rw**2 + v_rw**2)

    # gamma = psi - beta_rad - np.pi # angle between wind and ship in body
    gamma = - math.atan2(v_rw, u_rw)

    Cd_x = -Cd * np.cos(gamma)
    Cd_y = Cd * np.sin(gamma)
    Cd_n = Cd * np.sin(gamma*2) /5 # assuming 

    q = 0.5 * rho_air * U_wind_rel**2

    if U_wind_rel < 1e-6:
        # no wind
        Fx = 0.0
        Fy = 0.0
        Mz = 0.0
    else:
        Fx = q * A_f * Cd_x
        Fy = q * A_l * Cd_y
        Mz = q * A_l * Cd_n * Lpp
        
    return np.array([Fx, Fy, Mz])




In [ ]:
# 7) POST-PROCESSING & ANALYSIS
# ----------------------------------------------------------------------------------

def load_force_power_table():
    """
    Loads the CSV table with columns like [PWM, RPM, Current, Voltage, Power, Force, Efficiency].
    Returns an interpolation function: force_kgf -> power_W.
    The 'Force' column is in kgf, 'Power' is in W.
    """
    csv_path = "/Users/extra_m3/Desktop/SEAS_materials/Zagreb/Thesis/scripts/thruster_table/v12.csv"
    
    # 1) Load CSV (adjust decimal, separator if needed)
    df_table = pd.read_csv(csv_path, sep=';', decimal=',', header=0)
    
    # 2) Extract columns
    force_kgf = df_table[" Force "].values.astype(float)   # e.g. " Force "
    power_w   = df_table[" Power "].values.astype(float)   # e.g. " Power "
    
    # If your data has negative force for reverse, you can do:
    # force_kgf = np.abs(force_kgf)

    # 3) Build interpolation function
    power_lookup_func = interp1d(
        force_kgf,
        power_w,
        kind='linear',
        fill_value="extrapolate"
    )

    return power_lookup_func


def add_propulsion_power_column(df, power_lookup_func, sim_params, ship_params):
    """
    df: DataFrame with columns ["Fx_prop", "Fy_prop", "Mz_prop"] in SI units:
        - Fx_prop, Fy_prop in Newtons
        - Mz_prop in N·m
    power_lookup_func: function(force_kgf) -> power_W, from load_force_power_table
    sim_params: for dt
    ship_params: for Lpp

    Returns: (df_new, total_energy_joules)
        df_new is a copy of df with a new column "Propulsion_Power_W".
        total_energy_joules is the integral of that power over time.
    """
    Lpp = ship_params["Lpp"]
    dt  = sim_params["dt"]  # assumed constant

    df_new = df.copy()

    # 1) Net translational force
    Fx = df_new["Fx_prop"]
    Fy = df_new["Fy_prop"]
    F_trans = np.sqrt(Fx**2 + Fy**2)  # [N]

    # 2) Convert to kgf
    F_trans_kgf = F_trans / 9.81

    # 3) Interpolate for translational power
    Power_trans = power_lookup_func(F_trans_kgf)  # [W]

    # 4) Yaw force from Mz
    Mz = df_new["Mz_prop"]  # [N·m]
    Fz = np.abs(Mz) / Lpp    # [N]
    Fz_kgf = Fz / 9.81

    # 5) Interpolate for rotational power
    Power_rot = power_lookup_func(Fz_kgf)  # [W]

    # 6) Sum
    df_new["Propulsion_Power_W"] = Power_trans + Power_rot

    # 7) Integrate to get total energy (J)
    #    E = sum( P(t) * dt ) for each row
    total_energy_joules = (df_new["Propulsion_Power_W"] * dt).sum()

    return df_new, total_energy_joules



def analyze_anchor_motion(df, skip_frac=0.5, Lpp=1.5):
    """
    Analyze the tethered/anchored motion using the bow's position.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Must have columns: "t", "rode_length"
        If it doesn't have "bow_x" and "bow_y", we can compute them
        from (x, y, psi) and Lpp inside this function.
    skip_frac : float
        Fraction of the time series to skip from the start (to remove transient).
    Lpp : float
        Ship length between perpendiculars, needed if we must compute bow coords.
    
    Returns
    -------
    analysis : dict
        Keys include:
          "rode_length_amp"  : peak-to-peak amplitude of rode length
          "rode_length_freq" : dominant frequency of rode length (Hz)
          "swing_angle_amp"  : peak-to-peak anchor-swing angle
          "swing_angle_freq" : dominant frequency (Hz)
    """
    # 1) Determine the index after which we consider "steady" motion
    N = len(df)
    start_idx = int(skip_frac * N)
    df_steady = df.iloc[start_idx:].copy()

    # 2) If we didn't store bow coords or angle, compute them
    #    We'll assume you stored 'rode_length', but not 'bow_x' or 'bow_y'.
    #    If you did store them, skip this part.
    if "bow_x" not in df.columns or "bow_y" not in df.columns:
        # compute bow coords from (x, y, psi)
        x_vals = df_steady["x"].values
        y_vals = df_steady["y"].values
        psi_vals= df_steady["psi"].values
        x_bow = x_vals + 0.5 * Lpp * np.cos(psi_vals)
        y_bow = y_vals + 0.5 * Lpp * np.sin(psi_vals)

        df_steady["bow_x"] = x_bow
        df_steady["bow_y"] = y_bow
    else:
        # we already have them
        pass

    # 3) Extract or compute anchor angle
    if "bow_x" in df_steady.columns and "bow_y" in df_steady.columns:
        bow_x = df_steady["bow_x"].values
        bow_y = df_steady["bow_y"].values
        anchor_angle = np.arctan2(bow_y, bow_x)
        df_steady["anchor_angle"] = anchor_angle
    else:
        raise ValueError("Need bow_x, bow_y to compute anchor angle.")

    # 4) Rode length array, anchor angle array
    rode_len = df_steady["rode_length"].values
    angle_arr= df_steady["anchor_angle"].values
    t_vals   = df_steady["t"].values

    # 5) Peak-to-peak amplitude
    rode_len_amp  = rode_len.max()  - rode_len.min()
    swing_angle_amp = angle_arr.max() - angle_arr.min()

    # 6) Approx. frequency by FFT
    #    (If t is uniform spacing, dt ~ (t[-1]-t[0])/(len(t)-1))
    if len(t_vals) > 1:
        dt = (t_vals[-1] - t_vals[0]) / (len(t_vals)-1)
    else:
        dt = 1.0  # fallback

    def find_dominant_frequency(signal, dt):
        # Remove mean
        sig_centered = signal - np.mean(signal)
        fft_vals = np.fft.fft(sig_centered)
        freqs = np.fft.fftfreq(len(sig_centered), d=dt)

        # Only look at positive freq half
        half = len(freqs)//2
        freqs_pos = freqs[:half]
        fft_mag   = np.abs(fft_vals[:half])

        # index of largest amplitude peak (ignoring DC if you prefer)
        if len(freqs_pos) > 1:
            idx_max = np.argmax(fft_mag[1:]) + 1  # skip DC bin
            dom_freq = freqs_pos[idx_max]
        else:
            dom_freq = 0.0
        return dom_freq

    rode_len_freq = find_dominant_frequency(rode_len, dt)
    swing_angle_freq = find_dominant_frequency(angle_arr, dt)

    # 7) Convert angle amplitude to degrees
    swing_angle_amp_deg = np.degrees(swing_angle_amp)

    # 8) Compile results, with unit labels in dictionary keys
    analysis = {
        "rode_length_amp_m":  rode_len_amp,
        "rode_length_freq_hz": rode_len_freq,
        "swing_angle_amp_deg": swing_angle_amp_deg,
        "swing_angle_freq_hz": swing_angle_freq,
        "dt_s": dt
    }
    return analysis


In [ ]:
# 8) CONTROLLER & MPC setup

import casadi as ca
import do_mpc
import numpy as np


def anchor_force_casadi(x, y, psi):
    """
    Symbolic anchor force for the MPC model using a shifted power law:
    
        F(r) = 0                  if r <= r0
             = a*(r - r0)^b       if r > r0,
    
    where r is the distance from the anchor (global origin) to the bow.
    """
    Lpp = ship_params["Lpp"]
    xg = ship_params["xg"]
    
    # Bow coordinates in global frame.
    pa_x = x + 0.5 * Lpp * ca.cos(psi)
    pa_y = y + 0.5 * Lpp * ca.sin(psi)
    pa_tot = ca.sqrt(pa_x**2 + pa_y**2)
    
    # Get the power-law parameters from sim_params (as constants).
    a = ca.DM(sim_params.get("anchor_a", 0.0025))
    b = ca.DM(sim_params.get("anchor_b", 5.32))
    r0 = ca.DM(sim_params.get("anchor_r0", 3.0))
    
    # Compute the force magnitude using a shifted power law.
    # Option 1: Using if_else:
    F = ca.if_else(pa_tot <= r0, 0, a * (pa_tot - r0)**b)
    
    # Option 2: Alternatively, you can use a max function:
    # F = a * (ca.fmax(0, pa_tot - r0))**b
    
    # Compute the unit vector from bow to anchor.
    ex = pa_x / (pa_tot + 1e-6)
    ey = pa_y / (pa_tot + 1e-6)
    
    # Anchor force in global coordinates (points toward the anchor).
    F_anch_g = ca.vertcat(-F * ex, -F * ey)
    
    # Convert global force to body frame.
    R = ca.vertcat(
        ca.horzcat(ca.cos(psi), ca.sin(psi)),
        ca.horzcat(-ca.sin(psi), ca.cos(psi))
    )
    F_anch_s = R.T @ F_anch_g
    
    # Compute moment about CG (assuming force acts at the bow).
    Fanch_s_y = F_anch_s[1]
    F_anch_m = Fanch_s_y * (Lpp/2 - xg)
    
    return F_anch_s[0], F_anch_s[1], F_anch_m


def compute_wind_force_3dof_casadi(u, v, psi, ship_params, sim_params):
    """
    CasADi version of compute_wind_force_3dof.

    Parameters
    ----------
    u, v, psi : CasADi symbolic expressions
        Surge, sway, and heading (rad) in BODY or inertial sense
        (depending on your frame usage). Typically for do_mpc, these are states.
    ship_params : dict
        Contains numeric constants for the vessel. E.g.:
          "Lpp", "A_lat", "A_front", "Cd_wind", etc.
    sim_params : dict
        Contains numeric constants for environment. E.g.:
          "wind_speed", "wind_dir_deg", "rho_air".
        If you want them as symbolic parameters, handle differently.

    Returns
    -------
    (Fx, Fy, Mz) : CasADi expressions
        Symbolic expressions for the wind force and moment in BODY frame.
    """

    # 1) Extract numeric parameters from dictionaries
    U_wind   = sim_params.get("wind_speed", 0.0)   # [m/s] wind speed
    beta_deg = sim_params.get("wind_dir_deg", 0.0) # [deg] global wind heading
    rho_air  = sim_params.get("rho_air", 1.225)
    
    Cd  = ship_params.get("Cd_wind", 0.2)  # dimensionless drag coefficient
    A_l = ship_params.get("A_lat", 1.5)    # lateral above-water area [m^2]
    A_f = ship_params.get("A_front", 0.5)  # frontal area [m^2]
    Lpp = ship_params["Lpp"]               # length between perpendiculars [m]

    # 2) Convert wind direction to radians as numeric constant
    beta_rad = (ca.pi/180.0) * beta_deg   # fixed if beta_deg is numeric

    # 3) Relative wind velocity in ship's body axes (or local reference)
    #    This approach uses a simple expression: 
    #       u_rw = u - U_wind*cos(beta_rad - psi)
    #       v_rw = v - U_wind*sin(beta_rad - psi)
    #    The sign/angle logic depends on how you define frames.
    u_rw = u - U_wind * ca.cos(beta_rad - psi)
    v_rw = v - U_wind * ca.sin(beta_rad - psi)

    # 4) Magnitude of relative wind
    U_wind_rel = ca.sqrt(u_rw**2 + v_rw**2)

    # 5) Angle of relative wind in body. 
    #    In your original code, you set gamma = - atan2(v_rw, u_rw).
    gamma = - ca.atan2(v_rw, u_rw)

    # 6) Directional-based drag coefficients for x, y, and a "yaw" portion
    #    (simple example from your code)
    Cd_x = -Cd * ca.cos(gamma)     # negative sign implies direction
    Cd_y =  Cd * ca.sin(gamma)
    Cd_n =  Cd * ca.sin(2*gamma)/5 # artificial moment coefficient from sin(2*gamma)

    # 7) Dynamic pressure
    q = 0.5 * rho_air * (U_wind_rel**2)

    # 8) Conditional if the wind is "very small." In CasADi, use ca.if_else:
    small_wind = 1e-6
    Fx_expr = q * A_f * Cd_x
    Fy_expr = q * A_l * Cd_y
    Mz_expr= q * A_l * Cd_n * Lpp

    # If relative wind < small_wind, set all to zero
    Fx = ca.if_else(U_wind_rel < small_wind, 0, Fx_expr)
    Fy = ca.if_else(U_wind_rel < small_wind, 0, Fy_expr)
    Mz = ca.if_else(U_wind_rel < small_wind, 0, Mz_expr)

    return Fx, Fy, Mz



def create_model_mpc():
    """
    Builds and returns a do_mpc.Model object 
    that encapsulates the 3-DoF ship dynamics (x,y,psi,u,v,r).
    """
    model = do_mpc.model.Model('continuous') # create a continuous model object
    
    # States
    x_   = model.set_variable('_x', 'x')   # global x
    y_   = model.set_variable('_x', 'y')   # global y
    psi_ = model.set_variable('_x', 'psi') # heading
    u_   = model.set_variable('_x', 'u')   # surge
    v_   = model.set_variable('_x', 'v')   # sway
    r_   = model.set_variable('_x', 'r')   # yaw rate

    # Control inputs
    tau_x_   = model.set_variable('_u', 'tau_x')
    tau_y_   = model.set_variable('_u', 'tau_y')
    tau_psi_ = model.set_variable('_u', 'tau_psi')

    # Ship parameters
    m = ship_params["m"]
    Iz = ship_params["Iz"]
    xg = ship_params["xg"]
    
    Xudot = ship_params["Xudot"]
    Yvdot = ship_params["Yvdot"]
    Yrdot = ship_params["Yrdot"]
    Nvdot = ship_params["Nvdot"]
    Nrdot = ship_params["Nrdot"]

    Xu = ship_params["Xu"]
    Yv = ship_params["Yv"]
    Yr = ship_params["Yr"]
    Nv = ship_params["Nv"] 
    Nr = ship_params["Nr"]

    # Current velocity
    Uc = sim_params.get("current_velocity", 0.0)    # speed m/s

    beta_cur = (ca.pi/180.0) * sim_params.get("current_dir_deg", 0.0)   # direction from global x-axis
    Ucx_g = Uc * np.cos(beta_cur)
    Ucy_g = Uc * np.sin(beta_cur)
    u_c_b =  Ucx_g*ca.cos(psi_) + Ucy_g*ca.sin(psi_)
    v_c_b = -Ucx_g*ca.sin(psi_) + Ucy_g*ca.cos(psi_)
    
    # relative velocity
    u_r = u_ - u_c_b
    v_r = v_ - v_c_b

    # non-linear damping will be zero as suggested by Fossen 
    # in his book for small vessels/low speeds
    
    
    # Mass matrix
    M_mat = ca.vertcat(
        ca.horzcat(m - Xudot, 0, 0),
        ca.horzcat(0, m - Yvdot, m*xg - Yrdot),
        ca.horzcat(0, m*xg - Nvdot, Iz - Nrdot)
    )
    
    # Damping matrix
    D_lin = ca.vertcat(
        ca.horzcat(-Xu, 0,   0),
        ca.horzcat(0,   -Yv, -Yr),
        ca.horzcat(0,   -Nv, -Nr)
    )

    # Coriolis rigid body matrix
    C_rb = ca.vertcat(
            ca.horzcat(0, 0, -m*(xg*r_ + v_)),
            ca.horzcat(0, 0,  m*u_),
            ca.horzcat(m*(xg*r_ + v_), -m*u_, 0)
        )

    
    # Coriolis added mass matrix
    c13 = Yvdot*v_r + 0.5*(Nvdot + Yrdot)*r_
    c23 = -Xudot*u_r

    C_a = ca.vertcat(
            ca.horzcat(0, 0, c13),
            ca.horzcat(0, 0, c23),
            ca.horzcat(-c13, -c23, 0)
        )


    # # (Optional) parameters for wind or current if you want them as known inputs
    # F_wind_x_ = model.set_variable('_p', 'F_wind_x')
    # F_wind_y_ = model.set_variable('_p', 'F_wind_y')

    # Kinematics: eta_dot = J(psi)*nu
    J_psi = ca.vertcat(
        ca.horzcat(ca.cos(psi_), -ca.sin(psi_), 0),
        ca.horzcat(ca.sin(psi_),  ca.cos(psi_), 0),
        ca.horzcat(0, 0, 1)
    )
    eta_dot = J_psi @ ca.vertcat(u_, v_, r_)
    x_dot   = eta_dot[0]
    y_dot   = eta_dot[1]
    psi_dot = eta_dot[2]


    # Compute anchor force symbolically
    tau_anch_x, tau_anch_y, tau_anch_psi = anchor_force_casadi(x_, y_, psi_)
    tau_wind_x, tau_wind_y, tau_wind_psi = compute_wind_force_3dof_casadi(u_, v_, psi_, ship_params, sim_params)   


    # Total external forces: anchor + wind + control
    tau_ext_x = tau_anch_x + tau_wind_x + tau_x_
    tau_ext_y = tau_anch_y + tau_wind_y + tau_y_
    tau_ext_psi = tau_anch_psi + tau_wind_psi + tau_psi_

    tau_ext = ca.vertcat(tau_ext_x, tau_ext_y, tau_ext_psi)

    nu_ = ca.vertcat(u_, v_, r_)
    nu_rel_ = ca.vertcat(u_r, v_r, r_)

    nu_dot = ca.solve(M_mat, tau_ext - C_rb@nu_ - C_a@nu_rel_ - D_lin@nu_rel_)

    u_dot = nu_dot[0]
    v_dot = nu_dot[1]
    r_dot = nu_dot[2]

        # Parameters for previous control inputs
    tau_x_prev_ = model.set_variable('_p', 'tau_x_prev')  # Previous surge control
    tau_y_prev_ = model.set_variable('_p', 'tau_y_prev')  # Previous sway control
    tau_psi_prev_ = model.set_variable('_p', 'tau_psi_prev')  # Previous yaw control


        # set ODEs
    model.set_rhs('x',   x_dot)
    model.set_rhs('y',   y_dot)
    model.set_rhs('psi', psi_dot)
    model.set_rhs('u',   u_dot)
    model.set_rhs('v',   v_dot)
    model.set_rhs('r',   r_dot)

    model.setup()
    return model



def create_mpc_controller(model):
    """
    Creates and returns a do_mpc.controller.MPC object 
    that controls (tau_x, tau_y, tau_psi).
    """
    mpc = do_mpc.controller.MPC(model)
    setup_mpc = {
        'n_horizon': 30,    # example
        't_step': sim_params["dt"],      # must match sim_params["dt"]
        'state_discretization': 'collocation',
        'collocation_type': 'radau',
        'collocation_deg': 2, # 2nd order Radau 
        'collocation_ni': 1, # single-step collocation
        'store_full_solution': True,
    }
    mpc.set_param(**setup_mpc)

    mpc.set_param(nlpsol_opts={
    'ipopt.print_level': 0,  # Reduce output to speed up and keep console clean > from 5 to 0
    'ipopt.max_iter': 20,   # Reduce > from 3000 to 200 for faster convergence attempts
    'ipopt.tol': 1e-6       # Slightly relaxed tolerance 1e-8 > 1e-6
    })



# Old weights
    #   # Tuning weights for cost
    # Q_nu = np.diag([0,50,50])
    # Q_nu_terminal = np.diag([0,100,100])
    
    # Q_eta = np.diag([0.01,0.01,10])
    # # Q_eta_terminal = np.diag([0.01,0.01,10])
    # Q_tau = np.diag([0,0,0])
    
        # Tuning weights for cost
    Q_nu = np.diag([0,50,50])                      # VELOCITY
    Q_nu_terminal = np.diag([0,100,100])

    Q_eta = np.diag([0.01,0.01,10])                 # POSITION
    Q_eta_terminal = np.diag([0.01,0.01,10])
    Q_tau = np.diag([0.06,0.06,0.06])               # PROPULSION

    # Stage cost
    x_ = model.x['x']
    y_ = model.x['y']
    psi_ = model.x['psi']
    u_ = model.x['u']
    v_ = model.x['v']
    r_ = model.x['r']
    tau_x_ = model.u['tau_x']
    tau_y_ = model.u['tau_y']
    tau_psi_ = model.u['tau_psi']

        # Rate limits for control inputs
    max_rate_tau = 1.0  # [N/s] or [Nm/s], max rate of change for controls


    tau_x_prev_ = model.p['tau_x_prev']
    tau_y_prev_ = model.p['tau_y_prev']
    tau_psi_prev_ = model.p['tau_psi_prev']


    # Add rate-of-change constraints for each control input
    mpc.set_nl_cons('rate_limit_tau_x_pos', tau_x_ - tau_x_prev_ - max_rate_tau, ub=0.0)
    mpc.set_nl_cons('rate_limit_tau_x_neg', tau_x_prev_ - tau_x_ - max_rate_tau, ub=0.0)

    mpc.set_nl_cons('rate_limit_tau_y_pos', tau_y_ - tau_y_prev_ - max_rate_tau, ub=0.0)
    mpc.set_nl_cons('rate_limit_tau_y_neg', tau_y_prev_ - tau_y_ - max_rate_tau, ub=0.0)

    mpc.set_nl_cons('rate_limit_tau_psi_pos', tau_psi_ - tau_psi_prev_ - max_rate_tau, ub=0.0)
    mpc.set_nl_cons('rate_limit_tau_psi_neg', tau_psi_prev_ - tau_psi_ - max_rate_tau, ub=0.0)


    # Heading angle minimization
    psi_desired = ca.atan2(-y_, -x_)  # Angle toward the anchor point
    # Compute heading error
    psi_error = psi_ - psi_desired
    # Ensure continuity of the error (wrap to [-pi, pi])
    psi_error = ca.atan2(ca.sin(psi_error), ca.cos(psi_error))
    
    nu_vec = ca.vertcat(u_, v_, r_)
    eta_vec = ca.vertcat(x_, y_, psi_error)  # position + heading error
    tau_vec = ca.vertcat(tau_x_, tau_y_, tau_psi_)

    # For example, tracking x,y,psi to 0
    stage_cost =nu_vec.T@Q_nu@nu_vec + eta_vec.T@Q_eta@eta_vec + tau_vec.T@Q_tau@tau_vec
    terminal_cost = nu_vec.T@Q_nu_terminal@nu_vec # + eta_vec.T@Q_eta_terminal@eta_vec #+ tau_vec.T@Q_tau@tau_vec
    mpc.set_objective(lterm=stage_cost, mterm=terminal_cost)  # same terminal cost

    # Input bounds (example):
    max_tau = 10.0
    max_tau_psi = 5
    mpc.bounds['lower','_u','tau_x'] = -max_tau
    mpc.bounds['upper','_u','tau_x'] =  max_tau
    mpc.bounds['lower','_u','tau_y'] = -max_tau
    mpc.bounds['upper','_u','tau_y'] =  max_tau
    mpc.bounds['lower','_u','tau_psi'] = -max_tau_psi
    mpc.bounds['upper','_u','tau_psi'] =  max_tau_psi

    # Initial parameter values (set to 0 or any reasonable initial value)
    mpc.set_uncertainty_values(
        tau_x_prev=[0.0],
        tau_y_prev=[0.0],
        tau_psi_prev=[0.0]
    )
    mpc.setup()
    return mpc

In [ ]:
# 9) MAIN LOOP FUNCTION (MPC)
# ----------------------------------------------------------------------------------

def run_simulation_with_mpc(sim_params, ship_params, model, mpc):
    """
    Runs the simulation loop using do_mpc for control, 
    engaging the MPC only if the tension force exceeds a threshold.
    Otherwise, the propulsion force is zero.
    """
    # 1) Initialize
    dt = sim_params["dt"]
    t_max = sim_params["t_max"]
    n_steps = int(np.floor(t_max / dt))
    state0   = sim_params["initial_state"].copy()

    # Initial state array (matching do_mpc's states order!)
    # Example: [x, y, psi, u, v, r]
    state = state0

    tau_x_prev = 0.0  # Initial values
    tau_y_prev = 0.0
    tau_psi_prev = 0.0

    # Set do_mpc's "x0"
    mpc.x0 = state
    mpc.set_initial_guess()

    # Threshold for activating MPC
    tension_threshold = 6                       # [N]

    results = []
    t_sim = 0.0

        # Current in global frame - for plots
    Uc = sim_params.get("current_velocity", 0.0)    # speed m/s
    beta_deg = sim_params.get("current_dir_deg", 0.0)  # direction from x-axis
    beta_rad = np.deg2rad(beta_deg)
    # Global current components:
    Ucx_g = Uc * np.cos(beta_rad)
    Ucy_g = Uc * np.sin(beta_rad)

   # Store original wind parameters so that we can revert to them outside gusts.
    original_wind_speed = sim_params.get("wind_speed", 0.0)
    original_wind_dir   = sim_params.get("wind_dir_deg", 0.0)

 # Gust configuration parameters (with defaults provided).
    gust_period = sim_params.get("gust_period", 30)              # seconds between gusts
    gust_duration = sim_params.get("gust_duration", 5)             # gust lasts 5 seconds
    gust_speed_increase = sim_params.get("gust_speed_increase", 5)   # add 5 m/s during gust
    gust_angle_increase = sim_params.get("gust_angle_increase", 30)  # add 30° during gust


    for i in range(n_steps + 1):
        # Unpack for clarity
        x, y, psi, u, v, r = state

        mpc.set_uncertainty_values(tau_x_prev=[tau_x_prev], 
                            tau_y_prev=[tau_y_prev], 
                            tau_psi_prev=[tau_psi_prev])
        
  # --- Gust logic ---
        # Check if current simulation time falls within a gust window.
        # We use the modulo operator to determine if t_sim falls within the first 'gust_duration'
        # seconds of each gust period.
        if (t_sim >= t_max / 2) and ((t_sim % gust_period) < gust_duration):
            sim_params["wind_speed"] = original_wind_speed + gust_speed_increase
            sim_params["wind_dir_deg"] = original_wind_dir + gust_angle_increase
        else:
            sim_params["wind_speed"] = original_wind_speed
            sim_params["wind_dir_deg"] = original_wind_dir

            
        # 2) Compute anchor force
        tau_anchor, anchor_mag = compute_anchor_force(ship_params, sim_params, x, y, psi)
        tau_wind = compute_wind_force_3dof(u, v, psi, ship_params, sim_params)

        # # # 4) Engage MPC control based on tension force
        # if anchor_mag > tension_threshold:
        #     # Set external parameters (e.g., wind) for do_mpc
        #     # mpc.set_uncertainty_values(F_wind_x=[Fx_wind_g], F_wind_y=[Fy_wind_g])
        #     # Get control input from MPC
        #     u_opt = mpc.make_step(state)
        #     tau_prop = np.array(u_opt).flatten()  # [tau_x, tau_y, tau_psi]
                
        #         # Update previous inputs
        #     tau_x_prev = tau_prop[0]
        #     tau_y_prev = tau_prop[1]
        #     tau_psi_prev = tau_prop[2]

        # else:
        #     # No propulsion when tension is below the threshold
        #     tau_prop = np.array([0.0, 0.0, 0.0])

        tau_prop = np.array([0.0, 0.0, 0.0])




        # to add relative velocities to RESULTS 
        cos_psi = np.cos(psi)
        sin_psi = np.sin(psi)
        # R^T(psi) = [[ cos(psi), sin(psi)],
        #             [-sin(psi), cos(psi)]]
        u_c_b =  Ucx_g*cos_psi + Ucy_g*sin_psi
        v_c_b = -Ucx_g*sin_psi + Ucy_g*cos_psi

        # 4) Relative velocities
        u_r = u - u_c_b
        v_r = v - v_c_b


        # 5) Sum external forces in the body frame
        tau_ext = tau_anchor + tau_wind + tau_prop

        tau_wind_total = np.sqrt(tau_wind[0]**2 + tau_wind[1]**2)

        # Compute bow coords in GLOBAL
        Lpp = ship_params["Lpp"]
        x_bow = x + 0.5 * Lpp * np.cos(psi)
        y_bow = y + 0.5 * Lpp * np.sin(psi)

        # Distance from anchor (0,0)
        rode_length = np.hypot(x_bow, y_bow)

        

        # RK4 integration
        k1, nu_dot = compute_dstate(ship_params, sim_params, state, tau_ext)
        k2, _ = compute_dstate(ship_params, sim_params, state + 0.5*dt*k1, tau_ext)
        k3, _ = compute_dstate(ship_params, sim_params, state + 0.5*dt*k2, tau_ext)
        k4 , _ = compute_dstate(ship_params, sim_params, state + dt*k3,     tau_ext)
        state = state + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)


        # 7) Store logs
        results.append({
            "t": t_sim,
            "x": x, "y": y, "psi": psi,
            "u": u, "v": v, "r": r,
            "u_r": u_r,
            "v_r": v_r,
            "rode_length": rode_length,

            "Fx_prop": tau_prop[0],
            "Fy_prop": tau_prop[1],
            "Mz_prop": tau_prop[2],

            "Fx_wind": tau_wind[0],
            "Fy_wind": tau_wind[1],
            "Mz_wind": tau_wind[2],
            "wind_mag": tau_wind_total,

            "wind_speed": sim_params.get("wind_speed", 0.0),
            "wind_dir_deg": sim_params["wind_dir_deg"],
            
            "Fx_total_ext": tau_ext[0],
            "Fy_total_ext": tau_ext[1],
            "Mz_total_ext": tau_ext[2],

            "Fx_anchor": tau_anchor[0],
            "Fy_anchor": tau_anchor[1],
            "Mz_anchor": tau_anchor[2],
            "anchor_mag": anchor_mag,  # log the anchor tension for debugging
        })

        # Advance time
        t_sim += dt
    
    # Restore the original wind direction in sim_params.
    sim_params["wind_dir_deg"] = original_wind_dir

    # Convert results to a Pandas DataFrame
    df = pd.DataFrame(results)
    return df


In [ ]:
# 10) Create/Update MPC controller (MPC)
# ----------------------------------------------------------------------------------

# Create model and MPC
model_mpc = create_model_mpc()
mpc_controller = create_mpc_controller(model_mpc)


In [ ]:
# 11) RUN SIMULATION

start = time.perf_counter()
df = run_simulation_with_mpc(sim_params, ship_params, model_mpc, mpc_controller)
end = time.perf_counter()
elapsed = end - start

print(f"Simulation took {elapsed:.3f} seconds")


In [ ]:
# 12) POST PROCESSING - RESULTS

#1) Load the interpolation
power_lookup = load_force_power_table()
df_sim=df.copy()

# 2) Add power estimation
df_with_power, total_energy_j = add_propulsion_power_column(df_sim, power_lookup, sim_params, ship_params)
# 3) Anchor tension maximum values (global and stationary dropping 1/3 of df)
anchor_max_glob = np.sqrt(df["Fx_anchor"]**2 + df["Fy_anchor"]**2).max()

num_rows_to_drop = len(df_sim) // 4
# Drop portion of the rows
df_trimmed = df.iloc[-num_rows_to_drop:]
# Find the maximum anchor line tension value
anchor_max_1_3 = np.sqrt(df_trimmed["Fx_anchor"]**2 + df_trimmed["Fy_anchor"]**2).max()


analysis_dict = analyze_anchor_motion(df_with_power, skip_frac=0.5, Lpp=ship_params["Lpp"])

print("==============================================")
print(f"  ANCHOR MOTION ANALYSIS: SIMULATION RESULTS     w{sim_params.get('wind_speed')}c{sim_params.get('current_velocity')}ang{sim_params.get('wind_dir_deg')}")
print("----------------------------------------------")
print("          Simulated time:        External parameters:")
print(f"Simulation time: {sim_params['t_max']} s,           Wind speed: {sim_params.get('wind_speed', 0.0)} m/s")
print(f"Time step: {sim_params['dt']} s,                   Wind direction: {sim_params.get('wind_dir_deg', 0.0)} deg") 
print(f"Number of steps: {len(df)},           Current speed: {sim_params.get('current_velocity', 0.0)} m/s")
print(f"                                 Current direction: {sim_params.get('current_dir_deg', 0.0)} deg")


print("=======================================")
print("           Swing Angle:")

print(f"Swing angle amplitude: {analysis_dict['swing_angle_amp_deg']:.3f} deg")
print(f"Swing angle frequency: {analysis_dict['swing_angle_freq_hz']:.3f} Hz")
print(f"Swing angle oscillation period: {1.0/analysis_dict['swing_angle_freq_hz']:.3f} s")

print("---------------------------------------")
print("          Rode Length:")
print(f"Rode length amplitude: {analysis_dict['rode_length_amp_m']:.3f} m,      Maximal anchor force: {anchor_max_glob:.2f} N")
print(f"Rode length frequency: {analysis_dict['rode_length_freq_hz']:.3f} Hz,     Maximal anchor force (last 1/3): {anchor_max_1_3:.2f} N") 
print(f"Average distance to anchor point: {df_trimmed['rode_length'].mean():.3f} m,     Delta A Force: {anchor_max_glob-anchor_max_1_3:.2f} N")                                    
print("---------------------------------------")
print(f"Total Energy: {total_energy_j/1000:.2f} kJ")


In [ ]:
# # 13) POST-PROCESS PLOTS (A) - Trajectory plot
# # ----------------------------------------------------------------------------------

# A - Trajectory plot


# Extract current & wind info from sim_params
current_speed_m_s = sim_params.get("current_velocity", 0.0)
current_dir_deg   = sim_params.get("current_dir_deg", 0.0)

wind_speed_m_s    = sim_params.get("wind_speed", 0.0)  # or "wind_force_mag" if you have
wind_dir_deg      = sim_params.get("wind_dir_deg", 0.0)

# Convert angles to radians for trig
curr_rad = np.deg2rad(current_dir_deg)
wind_rad = np.deg2rad(wind_dir_deg)



# Suppose these fractions match your time-varying wind logic:
c1 = sim_params["c1"]  
c2 = sim_params["c2"]  
c3 = sim_params["c3"]  
c4 = sim_params["c4"]  
t_max = sim_params["t_max"]  # e.g. 100 s

# Segment the DataFrame by phase
df_phase1 = df[df["t"] <  c1 * t_max]
df_phase2 = df[(df["t"] >= c1 * t_max) & (df["t"] < c2 * t_max)]
df_phase3 = df[(df["t"] >= c2 * t_max) & (df["t"] < c3 * t_max)]
df_phase4 = df[(df["t"] >= c3 * t_max) & (df["t"] < c4 * t_max)]
df_phase5 = df[df["t"] >= c4 * t_max]

# Create the figure and axis
fig, ax = plt.subplots(figsize=(15,15))

# 1) Plot each phase of the trajectory in a different color
ax.plot(df_phase1["x"], df_phase1["y"], color="blue",  linestyle="--", label="Phase 1")
ax.plot(df_phase2["x"], df_phase2["y"], color="orange", linestyle="--", label="Phase 2")
ax.plot(df_phase3["x"], df_phase3["y"], color="red",    linestyle="--", label="Phase 3")
ax.plot(df_phase4["x"], df_phase4["y"], color="green",  linestyle="--", label="Phase 4")
ax.plot(df_phase5["x"], df_phase5["y"], color="purple", linestyle="--", label="Phase 5")

# 2) Mark the start point (blue), smaller dot
x_start = df["x"].iloc[0]
y_start = df["y"].iloc[0]
ax.scatter(x_start, y_start, color="blue", s=20, zorder=5, label="Start Point")

# 3) Mark the end point (green), smaller dot
x_end = df["x"].iloc[-1]
y_end = df["y"].iloc[-1]
ax.scatter(x_end, y_end, color="green", s=20, zorder=5, label="End Point")

# 4) Draw a dashed circle around anchor at (0,0) with radius=1
anchor_circle = Circle(
    (0, 0),       # center
    radius=1,     # 1 meter
    color="black",
    fill=False,   # don't fill
    linestyle="--",
    linewidth=1
)
ax.add_patch(anchor_circle)

# 5) Plot snapshot ship arrows every 1/20th of t_max (including final).
#    Also draw a thin dotted line to (0,0) from the bow (anchor line).
num_arrows = 20
dt_arrow   = t_max / num_arrows

for i in range(num_arrows):
    t_snap = i * dt_arrow
    # Find the index in df that is closest to this time
    idx = (df["t"] - t_snap).abs().idxmin()
    
    x_i   = df["x"].iloc[idx]
    y_i   = df["y"].iloc[idx]
    psi_i = df["psi"].iloc[idx]
    
    # Compute stern and bow
    Lpp = ship_params["Lpp"]
    x_stern = x_i - 0.5 * Lpp * np.cos(psi_i)
    y_stern = y_i - 0.5 * Lpp * np.sin(psi_i)
    x_bow   = x_i + 0.5 * Lpp * np.cos(psi_i)
    y_bow   = y_i + 0.5 * Lpp * np.sin(psi_i)
    
    # Thin snapshot arrow
    arrow_thin = FancyArrowPatch(
        (x_stern, y_stern), (x_bow, y_bow),
        mutation_scale=15,
        color='black',
        arrowstyle='-|>',
        linewidth=1,   # thinner arrow
        alpha=0.7
    )
    ax.add_patch(arrow_thin)



# 6) Draw final ship arrow (thicker) & anchor line
psi_end  = df["psi"].iloc[-1]
Lpp      = ship_params["Lpp"]
x_stern  = x_end - 0.5 * Lpp * np.cos(psi_end)
y_stern  = y_end - 0.5 * Lpp * np.sin(psi_end)
x_bow    = x_end + 0.5 * Lpp * np.cos(psi_end)
y_bow    = y_end + 0.5 * Lpp * np.sin(psi_end)

arrow_final = FancyArrowPatch(
    (x_stern, y_stern), (x_bow, y_bow),
    mutation_scale=20,
    color='red',
    arrowstyle='-|>',
    linewidth=2
)
ax.add_patch(arrow_final)

# Also draw final anchor line
ax.plot([0, x_bow], [0, y_bow],
        color="black", ls=":", lw=1, alpha=0.7)

# 7) Axis labels, title, legend, and grid
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
ax.set_aspect("equal", adjustable="box")  # maintain aspect ratio



ax.set_title("Ship Trajectory with Time-Varying Wind")
ax.grid(True)


# --- 8) Add ARROWS for Current and Wind in top-right corner
# Choose a corner coordinate near the top-right
corner_x = 0.7
corner_y = -3

# We'll define a scaling factor so the arrow is visible:
arrow_scale = 1.0  # adjust as needed

# Current arrow
cx_end = corner_x + arrow_scale * np.cos(curr_rad)
cy_end = corner_y + arrow_scale * np.sin(curr_rad)

arrow_current = FancyArrowPatch(
    (corner_x, corner_y),
    (cx_end, cy_end),
    mutation_scale=15,
    color="blue",
    arrowstyle='-|>',
    linewidth=2,
    label=f"Current: {current_speed_m_s:.2f} m/s @ {current_dir_deg:.1f}°"
)
ax.add_patch(arrow_current)

# Now shift the wind arrow slightly below that
wind_x0 = 0.6
wind_y0 = -3.8 # just shift down by 1 for clarity

wx_end = wind_x0 + arrow_scale * np.cos(wind_rad)
wy_end = wind_y0 + arrow_scale * np.sin(wind_rad)

arrow_wind = FancyArrowPatch(
    (wind_x0, wind_y0),
    (wx_end, wy_end),
    mutation_scale=15,
    color="green",
    arrowstyle='-|>',
    linewidth=2,
    label=f"Wind: {wind_speed_m_s:.2f} m/s @ {wind_dir_deg:.1f}°"
)
ax.add_patch(arrow_wind)

# 9) Re-build the legend so arrows appear with labels
ax.legend(loc="best")

# Fix axis limits 
ax.set_xlim([-2, 13])
ax.set_ylim([-4, 8])

plt.show()



In [ ]:
# 14) POST PROCESSING -  time history plots
# 
def plot_wind_forces(df):
    # Compute wind magnitude
    wind_mag = np.sqrt(df["Fx_wind"]**2 + df["Fy_wind"]**2)

    # Create a single-axes figure
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df["t"], 
        y=df["Fx_wind"], 
        mode='lines', 
        name='Fx wind',
        line=dict(color='blue')
    ))
    fig.add_trace(go.Scatter(
        x=df["t"], 
        y=df["Fy_wind"], 
        mode='lines', 
        name='Fy wind',
        line=dict(color='orange')
    ))
    fig.add_trace(go.Scatter(
        x=df["t"], 
        y=wind_mag, 
        mode='lines', 
        name='Wind magnitude',
        line=dict(color='green')
    ))

    fig.update_layout(
        title="Time History: Wind Forces",
        xaxis_title="Time (s)",
        yaxis_title="Wind Force (N)",
        template="plotly_white",
        
        width=1200,  # Adjust the width as needed
        height=600   # Adjust the height as needed
    )
    
    
    fig.show()

def plot_anchor_forces(df):
    # Compute anchor force magnitude
    anchor_mag = np.sqrt(df["Fx_anchor"]**2 + df["Fy_anchor"]**2)

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.update_layout(
        title="Time History: Anchor Forces & Moment",
        template="plotly_white",
        width=1200,  # Adjust the width as needed
        height=600   # Adjust the height as needed
    )

    # Primary y-axis => anchor forces
    fig.add_trace(go.Scatter(
        x=df["t"], 
        y=df["Fx_anchor"], 
        mode='lines',
        name='Fx anchor',
        line=dict(color='blue')
    ), secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df["t"], 
        y=df["Fy_anchor"], 
        mode='lines',
        name='Fy anchor',
        line=dict(color='orange')
    ), secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df["t"], 
        y=anchor_mag, 
        mode='lines',
        name='Anchor magnitude',
        line=dict(color='green')
    ), secondary_y=False)

    # Secondary y-axis => Mz_anchor
    fig.add_trace(go.Scatter(
        x=df["t"], 
        y=df["Mz_anchor"], 
        mode='lines',
        name='Mz anchor',
        line=dict(color='red')
    ), secondary_y=True)

    # Update axis titles
    fig.update_xaxes(title_text="Time (s)")
    fig.update_yaxes(title_text="Anchor Force (N)", secondary_y=False)
    fig.update_yaxes(title_text="Anchor Moment (Nm)", secondary_y=True)

    fig.show()

def plot_prop_forces(df):
    # Compute prop force magnitude
    prop_mag = np.sqrt(df["Fx_prop"]**2 + df["Fy_prop"]**2)

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.update_layout(
        title="Time History: Propulsion Forces & Moment",
        template="plotly_white",
                width=1200,  # Adjust the width as needed
        height=600   # Adjust the height as needed
    )

    # Primary y-axis => propulsion forces
    fig.add_trace(go.Scatter(
        x=df["t"], 
        y=df["Fx_prop"], 
        mode='lines',
        name='Fx prop',
        line=dict(color='blue')
    ), secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df["t"], 
        y=df["Fy_prop"], 
        mode='lines',
        name='Fy prop',
        line=dict(color='orange')
    ), secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df["t"], 
        y=prop_mag, 
        mode='lines',
        name='Prop magnitude',
        line=dict(color='green')
    ), secondary_y=False)

    # Secondary y-axis => Mz_prop
    fig.add_trace(go.Scatter(
        x=df["t"], 
        y=df["Mz_prop"], 
        mode='lines',
        name='Mz prop',
        line=dict(color='red')
    ), secondary_y=True)

    fig.update_xaxes(title_text="Time (s)")
    fig.update_yaxes(title_text="Propulsion Force (N)", secondary_y=False)
    fig.update_yaxes(title_text="Propulsion Moment (Nm)", secondary_y=True)
    fig.show()


def plot_wind_forces(df):
    # Compute wind force magnitude
    wind_mag = np.sqrt(df["Fx_wind"]**2 + df["Fy_wind"]**2)

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.update_layout(
        title="Time History: Wind Forces & Moment",
        template="plotly_white",
                width=1200,  # Adjust the width as needed
        height=600   # Adjust the height as needed
    )

    # Primary y-axis => wind forces
    fig.add_trace(go.Scatter(
        x=df["t"], 
        y=df["Fx_wind"], 
        mode='lines',
        name='Fx wind',
        line=dict(color='blue')
    ), secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df["t"], 
        y=df["Fy_wind"], 
        mode='lines',
        name='Fy wind',
        line=dict(color='orange')
    ), secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df["t"], 
        y=wind_mag, 
        mode='lines',
        name='Wind magnitude',
        line=dict(color='green')
    ), secondary_y=False)

    # # Secondary y-axis => Mz_wind
    fig.add_trace(go.Scatter(
        x=df["t"], 
        y=df["Mz_wind"], 
        mode='lines',
        name='Mz wind',
        line=dict(color='red')
    ), secondary_y=True)

    fig.update_xaxes(title_text="Time (s)")
    fig.update_yaxes(title_text="Wind Force (N)", secondary_y=False)
    # fig.update_yaxes(title_text="Wind Moment (Nm)", secondary_y=True)

    fig.show()


def plot_velocities(df):
    vel_mag = np.sqrt(df["u"]**2 + df["v"]**2)

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.update_layout(
        title="Ship Velocities & Angular Rate",
        template="plotly_white",
                width=1200,  # Adjust the width as needed
        height=600   # Adjust the height as needed
    )

    # Primary axis => u, v, and velocity magnitude
    fig.add_trace(go.Scatter(
        x=df["t"],
        y=df["u"],
        mode='lines',
        name='u (surge)',
        line=dict(color='blue')
    ), secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df["t"],
        y=df["v"],
        mode='lines',
        name='v (sway)',
        line=dict(color='orange')
    ), secondary_y=False)

    fig.add_trace(go.Scatter(
        x=df["t"],
        y=vel_mag,
        mode='lines',
        name='Velocity magnitude',
        line=dict(color='green')
    ), secondary_y=False)

    # Secondary axis => r
    fig.add_trace(go.Scatter(
        x=df["t"],
        y=df["r"],
        mode='lines',
        name='r (yaw rate)',
        line=dict(color='red')
    ), secondary_y=True)

    fig.update_xaxes(title_text="Time (s)")
    fig.update_yaxes(title_text="Linear Velocity (m/s)", secondary_y=False)
    fig.update_yaxes(title_text="Yaw Rate (rad/s)", secondary_y=True)

    fig.show()


# plot_wind_forces(df)
plot_anchor_forces(df)
plot_prop_forces(df)
plot_wind_forces(df)
plot_velocities(df)

In [ ]:
# 15) POST PROCESSING -  propulsion power plot
# Create a line plot
plt.figure(figsize=(10, 6))
plt.plot(df_with_power['t'], df_with_power['Propulsion_Power_W'], color='b', label='Propulsion Power (W)')

# Add labels and title
plt.xlabel('Time (s)')
plt.ylabel('Propulsion Power (W)')
plt.title('Propulsion Power over Time')
plt.legend()
plt.grid(True)


# Show the plot
plt.show()

In [ ]:
# Animation 

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch, Circle
from matplotlib.animation import FuncAnimation
from matplotlib import rc

# So that animations display in Jupyter as JavaScript (optional):
rc('animation', html='jshtml', embed_limit=30 * 1024 * 1024)

def animate_simulation(df, ship_params, fps=20, speedup=1.0):
    """
    Animate the ship's movement from a DataFrame 'df' containing columns:
        "t", "x", "y", "psi", (optionally "Fx_wind", "Fy_wind") etc.
    The animation includes:
        - A ship arrow (bow/stern)
        - A trail of past positions
        - A time text
        - A line from anchor to bow
        - An optional wind arrow (if you provide wind logic)
        - An anchor circle at the origin
    
    Parameters
    ----------
    df : pd.DataFrame
        Must have columns ["t", "x", "y", "psi"] at minimum.
    ship_params : dict
        Must include "Lpp" for the length between perpendiculars, used to draw the ship arrow.
    fps : int, optional
        Frames per second for the animation. Default 20.
    speedup : float, optional
        Factor to speed up (or slow down) the animation vs. real time.
        e.g. speedup=2.0 => plays 2x faster than the real dt.
    """
    # Extract arrays from the DataFrame
    time = df["t"].values
    x = df["x"].values
    y = df["y"].values
    psi = df["psi"].values
    
    Lpp = ship_params["Lpp"]
    
    # Figure & Axis
    fig, ax = plt.subplots(figsize=(7, 7))
    
    # Draw an anchor circle (radius=1 m)
    anchor_circle = Circle((0,0), radius=1, color='k', fill=False, linestyle='--', linewidth=1)
    ax.add_patch(anchor_circle)
    
    # Initial plot settings (axis limits, aspect)
    margin = 1.0
    x_min, x_max = np.min(x), np.max(x)
    y_min, y_max = np.min(y), np.max(y)
    ax.set_xlim(x_min - margin, x_max + margin)
    ax.set_ylim(y_min - margin, y_max + margin)
    ax.set_aspect('equal', adjustable='box')
    ax.set_title('Ship Movement Animation')
    ax.set_xlabel('X Position (m)')
    ax.set_ylabel('Y Position (m)')
    
    # --- 1) Initialize the ship arrow
    ship_arrow = FancyArrowPatch(
        (0, 0), (0, 0),
        mutation_scale=20,
        color='blue',
        arrowstyle='-|>',
        linewidth=2
    )
    ax.add_patch(ship_arrow)

    # --- 2) Initialize a trail line
    trail, = ax.plot([], [], 'b--', lw=1)

    # --- 3) Text object to display time
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

    # --- 4) Line from anchor point to bow
    anchor_line, = ax.plot([], [], 'k-', lw=1)


    # Determine the time step (assuming uniform) and set animation interval
    # dt ~ time[i+1] - time[i]
    if len(time) > 1:
        dt = (time[-1] - time[0]) / (len(time) - 1)
    else:
        dt = 0.1  # fallback

    # Real-time in ms per frame, but scaled by 'speedup'
    # e.g. if dt=0.1 s, fps=20 => each step is 0.1 s => 10x real speed if we show every sample
    frame_interval_ms = 1000 * dt / speedup
    
    # The init function: clear everything
    def init():
        ship_arrow.set_positions((0,0), (0,0))
        trail.set_data([], [])
        anchor_line.set_data([], [])
        time_text.set_text('')
        return ship_arrow, trail, anchor_line, time_text

    # The update function: for each frame i
    def animate(i):

        
        # Bow/stern
        Lpp_half = Lpp / 2.0
        cos_psi = np.cos(psi[i])
        sin_psi = np.sin(psi[i])

        x_bow = x[i] + Lpp_half * cos_psi
        y_bow = y[i] + Lpp_half * sin_psi
        x_stern = x[i] - Lpp_half * cos_psi
        y_stern = y[i] - Lpp_half * sin_psi

        # Update ship arrow
        ship_arrow.set_positions((x_stern, y_stern), (x_bow, y_bow))

        # Update trail
        trail.set_data(x[:i+1], y[:i+1])

        # Update anchor line
        anchor_line.set_data([0, x_bow], [0, y_bow])

        # Update time text
        time_text.set_text(f'Time: {time[i]:.2f} s')



        # Return all artists that changed
        return ship_arrow, trail, anchor_line, time_text

    # Create the animation
    ani = FuncAnimation(
        fig,
        animate,
        frames=len(time),
        init_func=init,
        interval=frame_interval_ms,  # how many ms between frames
        blit=False,                  # if True, must return *exact* artists
        repeat=True
    )

    # Display the animation in Jupyter
    return ani

ani = animate_simulation(df, ship_params, fps=20, speedup=2.0)
ani  # in Jupyter, displays the animation inline

In [ ]:
#DUMP

# Main loop with wind single wind direction change

# 6) MAIN LOOP ALTERNATIVE (MPC)
# ----------------------------------------------------------------------------------

def run_simulation_with_mpc(sim_params, ship_params, model, mpc):
    """
    Runs the simulation loop using do_mpc for control, 
    engaging the MPC only if the tension force exceeds a threshold.
    Otherwise, the propulsion force is zero.
    """
    # 1) Initialize
    dt = sim_params["dt"]
    t_max = sim_params["t_max"]
    n_steps = int(np.floor(t_max / dt))
    state0   = sim_params["initial_state"].copy()

    # Initial state array (matching do_mpc's states order!)
    # Example: [x, y, psi, u, v, r]
    state = state0

    tau_x_prev = 0.0  # Initial values
    tau_y_prev = 0.0
    tau_psi_prev = 0.0

    # Set do_mpc's "x0"
    mpc.x0 = state
    mpc.set_initial_guess()

    # Threshold for activating MPC
    tension_threshold = 6                       # [N]

    results = []
    t_sim = 0.0

        # Current in global frame - for plots
    Uc = sim_params.get("current_velocity", 0.0)    # speed m/s
    beta_deg = sim_params.get("current_dir_deg", 0.0)  # direction from x-axis
    beta_rad = np.deg2rad(beta_deg)
    # Global current components:
    Ucx_g = Uc * np.cos(beta_rad)
    Ucy_g = Uc * np.sin(beta_rad)

    # Store the original wind direction so it can be restored later.
    original_wind_dir = sim_params.get("wind_dir_deg", 0.0)
    wind_updated = False  # flag to ensure update is done only once


    for i in range(n_steps + 1):
        # Unpack for clarity
        x, y, psi, u, v, r = state

        mpc.set_uncertainty_values(tau_x_prev=[tau_x_prev], 
                            tau_y_prev=[tau_y_prev], 
                            tau_psi_prev=[tau_psi_prev])
        
        if (t_sim >= t_max / 2) and (not wind_updated):
            sim_params["wind_dir_deg"] = original_wind_dir + 30
            wind_updated = True
            
        # 2) Compute anchor force
        tau_anchor, anchor_mag = compute_anchor_force(ship_params, sim_params, x, y, psi)
        tau_wind = compute_wind_force_3dof(u, v, psi, ship_params, sim_params)

        # 4) Engage MPC control based on tension force
        if anchor_mag > tension_threshold:
            # Set external parameters (e.g., wind) for do_mpc
            # mpc.set_uncertainty_values(F_wind_x=[Fx_wind_g], F_wind_y=[Fy_wind_g])
            # Get control input from MPC
            u_opt = mpc.make_step(state)
            tau_prop = np.array(u_opt).flatten()  # [tau_x, tau_y, tau_psi]
                
                # Update previous inputs
            tau_x_prev = tau_prop[0]
            tau_y_prev = tau_prop[1]
            tau_psi_prev = tau_prop[2]

        else:
            # No propulsion when tension is below the threshold
            tau_prop = np.array([0.0, 0.0, 0.0])

        # tau_prop = np.array([0.0, 0.0, 0.0])




        # to add relative velocities to RESULTS 
        cos_psi = np.cos(psi)
        sin_psi = np.sin(psi)
        # R^T(psi) = [[ cos(psi), sin(psi)],
        #             [-sin(psi), cos(psi)]]
        u_c_b =  Ucx_g*cos_psi + Ucy_g*sin_psi
        v_c_b = -Ucx_g*sin_psi + Ucy_g*cos_psi

        # 4) Relative velocities
        u_r = u - u_c_b
        v_r = v - v_c_b


        # 5) Sum external forces in the body frame
        tau_ext = tau_anchor + tau_wind + tau_prop

        tau_wind_total = np.sqrt(tau_wind[0]**2 + tau_wind[1]**2)

        # Compute bow coords in GLOBAL
        Lpp = ship_params["Lpp"]
        x_bow = x + 0.5 * Lpp * np.cos(psi)
        y_bow = y + 0.5 * Lpp * np.sin(psi)

        # Distance from anchor (0,0)
        rode_length = np.hypot(x_bow, y_bow)

        

        # RK4 integration
        k1, nu_dot = compute_dstate(ship_params, sim_params, state, tau_ext)
        k2, _ = compute_dstate(ship_params, sim_params, state + 0.5*dt*k1, tau_ext)
        k3, _ = compute_dstate(ship_params, sim_params, state + 0.5*dt*k2, tau_ext)
        k4 , _ = compute_dstate(ship_params, sim_params, state + dt*k3,     tau_ext)
        state = state + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)


        # 7) Store logs
        results.append({
            "t": t_sim,
            "x": x, "y": y, "psi": psi,
            "u": u, "v": v, "r": r,
            "u_r": u_r,
            "v_r": v_r,
            "rode_length": rode_length,

            "Fx_prop": tau_prop[0],
            "Fy_prop": tau_prop[1],
            "Mz_prop": tau_prop[2],

            "Fx_wind": tau_wind[0],
            "Fy_wind": tau_wind[1],
            "Mz_wind": tau_wind[2],
            "wind_mag": tau_wind_total,
            
            "Fx_total_ext": tau_ext[0],
            "Fy_total_ext": tau_ext[1],
            "Mz_total_ext": tau_ext[2],

            "Fx_anchor": tau_anchor[0],
            "Fy_anchor": tau_anchor[1],
            "Mz_anchor": tau_anchor[2],
            "anchor_mag": anchor_mag,  # log the anchor tension for debugging
        })

        # Advance time
        t_sim += dt
    
    # Restore the original wind direction in sim_params.
    sim_params["wind_dir_deg"] = original_wind_dir

    # Convert results to a Pandas DataFrame
    df = pd.DataFrame(results)
    return df



# wind direction change +30 degrees each gust


# 6) MAIN LOOP ALTERNATIVE (MPC)
# ----------------------------------------------------------------------------------

def run_simulation_with_mpc(sim_params, ship_params, model, mpc):
    """
    Runs the simulation loop using do_mpc for control, 
    engaging the MPC only if the tension force exceeds a threshold.
    Otherwise, the propulsion force is zero.
    """
    # 1) Initialize
    dt = sim_params["dt"]
    t_max = sim_params["t_max"]
    n_steps = int(np.floor(t_max / dt))
    state0   = sim_params["initial_state"].copy()

    # Initial state array (matching do_mpc's states order!)
    # Example: [x, y, psi, u, v, r]
    state = state0

    tau_x_prev = 0.0  # Initial values
    tau_y_prev = 0.0
    tau_psi_prev = 0.0

    # Set do_mpc's "x0"
    mpc.x0 = state
    mpc.set_initial_guess()

    # Threshold for activating MPC
    tension_threshold = 6                       # [N]

    results = []
    t_sim = 0.0

        # Current in global frame - for plots
    Uc = sim_params.get("current_velocity", 0.0)    # speed m/s
    beta_deg = sim_params.get("current_dir_deg", 0.0)  # direction from x-axis
    beta_rad = np.deg2rad(beta_deg)
    # Global current components:
    Ucx_g = Uc * np.cos(beta_rad)
    Ucy_g = Uc * np.sin(beta_rad)

    # Store the original wind direction so it can be restored later.
    original_wind_dir = sim_params.get("wind_dir_deg", 0.0)
    # wind_updated = False  # flag to ensure update is done only once


    for i in range(n_steps + 1):
        # Unpack for clarity
        x, y, psi, u, v, r = state

        mpc.set_uncertainty_values(tau_x_prev=[tau_x_prev], 
                            tau_y_prev=[tau_y_prev], 
                            tau_psi_prev=[tau_psi_prev])
        
  # --- Wind burst logic ---
        # A burst occurs every 30 seconds for a duration of 5 seconds,
        # but only for t_sim >= 30 s.
        # The burst index is computed as floor(t_sim/30). For example:
        #   For t in [30,35): burst_index = 1, so wind_dir_deg = original + 30.
        #   For t in [60,65): burst_index = 2, so wind_dir_deg = original + 60.
        if t_sim >= 30 and (t_sim % 30) < 5:
            burst_index = int(t_sim // 30)
            sim_params["wind_dir_deg"] = original_wind_dir + 30 * burst_index
        else:
            sim_params["wind_dir_deg"] = original_wind_dir

            
        # 2) Compute anchor force
        tau_anchor, anchor_mag = compute_anchor_force(ship_params, sim_params, x, y, psi)
        tau_wind = compute_wind_force_3dof(u, v, psi, ship_params, sim_params)

        # 4) Engage MPC control based on tension force
        if anchor_mag > tension_threshold:
            # Set external parameters (e.g., wind) for do_mpc
            # mpc.set_uncertainty_values(F_wind_x=[Fx_wind_g], F_wind_y=[Fy_wind_g])
            # Get control input from MPC
            u_opt = mpc.make_step(state)
            tau_prop = np.array(u_opt).flatten()  # [tau_x, tau_y, tau_psi]
                
                # Update previous inputs
            tau_x_prev = tau_prop[0]
            tau_y_prev = tau_prop[1]
            tau_psi_prev = tau_prop[2]

        else:
            # No propulsion when tension is below the threshold
            tau_prop = np.array([0.0, 0.0, 0.0])

        # tau_prop = np.array([0.0, 0.0, 0.0])




        # to add relative velocities to RESULTS 
        cos_psi = np.cos(psi)
        sin_psi = np.sin(psi)
        # R^T(psi) = [[ cos(psi), sin(psi)],
        #             [-sin(psi), cos(psi)]]
        u_c_b =  Ucx_g*cos_psi + Ucy_g*sin_psi
        v_c_b = -Ucx_g*sin_psi + Ucy_g*cos_psi

        # 4) Relative velocities
        u_r = u - u_c_b
        v_r = v - v_c_b


        # 5) Sum external forces in the body frame
        tau_ext = tau_anchor + tau_wind + tau_prop

        tau_wind_total = np.sqrt(tau_wind[0]**2 + tau_wind[1]**2)

        # Compute bow coords in GLOBAL
        Lpp = ship_params["Lpp"]
        x_bow = x + 0.5 * Lpp * np.cos(psi)
        y_bow = y + 0.5 * Lpp * np.sin(psi)

        # Distance from anchor (0,0)
        rode_length = np.hypot(x_bow, y_bow)

        

        # RK4 integration
        k1, nu_dot = compute_dstate(ship_params, sim_params, state, tau_ext)
        k2, _ = compute_dstate(ship_params, sim_params, state + 0.5*dt*k1, tau_ext)
        k3, _ = compute_dstate(ship_params, sim_params, state + 0.5*dt*k2, tau_ext)
        k4 , _ = compute_dstate(ship_params, sim_params, state + dt*k3,     tau_ext)
        state = state + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)


        # 7) Store logs
        results.append({
            "t": t_sim,
            "x": x, "y": y, "psi": psi,
            "u": u, "v": v, "r": r,
            "u_r": u_r,
            "v_r": v_r,
            "rode_length": rode_length,

            "Fx_prop": tau_prop[0],
            "Fy_prop": tau_prop[1],
            "Mz_prop": tau_prop[2],

            "Fx_wind": tau_wind[0],
            "Fy_wind": tau_wind[1],
            "Mz_wind": tau_wind[2],
            "wind_mag": tau_wind_total,

            "wind_speed": sim_params.get("wind_speed", 0.0),
            "wind_dir_deg": sim_params["wind_dir_deg"],
            
            "Fx_total_ext": tau_ext[0],
            "Fy_total_ext": tau_ext[1],
            "Mz_total_ext": tau_ext[2],

            "Fx_anchor": tau_anchor[0],
            "Fy_anchor": tau_anchor[1],
            "Mz_anchor": tau_anchor[2],
            "anchor_mag": anchor_mag,  # log the anchor tension for debugging
        })

        # Advance time
        t_sim += dt
    
    # Restore the original wind direction in sim_params.
    sim_params["wind_dir_deg"] = original_wind_dir

    # Convert results to a Pandas DataFrame
    df = pd.DataFrame(results)
    return df




In [ ]:

# wind direction change +30 degrees each gust


# 6) MAIN LOOP ALTERNATIVE (MPC)
# ----------------------------------------------------------------------------------

def run_simulation_with_mpc(sim_params, ship_params, model, mpc):
    """
    Runs the simulation loop using do_mpc for control, 
    engaging the MPC only if the tension force exceeds a threshold.
    Otherwise, the propulsion force is zero.
    """
    # 1) Initialize
    dt = sim_params["dt"]
    t_max = sim_params["t_max"]
    n_steps = int(np.floor(t_max / dt))
    state0   = sim_params["initial_state"].copy()

    # Initial state array (matching do_mpc's states order!)
    # Example: [x, y, psi, u, v, r]
    state = state0

    tau_x_prev = 0.0  # Initial values
    tau_y_prev = 0.0
    tau_psi_prev = 0.0

    # Set do_mpc's "x0"
    mpc.x0 = state
    mpc.set_initial_guess()

    # Threshold for activating MPC
    tension_threshold = 6                       # [N]

    results = []
    t_sim = 0.0

        # Current in global frame - for plots
    Uc = sim_params.get("current_velocity", 0.0)    # speed m/s
    beta_deg = sim_params.get("current_dir_deg", 0.0)  # direction from x-axis
    beta_rad = np.deg2rad(beta_deg)
    # Global current components:
    Ucx_g = Uc * np.cos(beta_rad)
    Ucy_g = Uc * np.sin(beta_rad)

    # Store the original wind direction so it can be restored later.
    original_wind_dir = sim_params.get("wind_dir_deg", 0.0)
    # wind_updated = False  # flag to ensure update is done only once


    for i in range(n_steps + 1):
        # Unpack for clarity
        x, y, psi, u, v, r = state

        mpc.set_uncertainty_values(tau_x_prev=[tau_x_prev], 
                            tau_y_prev=[tau_y_prev], 
                            tau_psi_prev=[tau_psi_prev])
        
  # --- Wind burst logic ---
        # A burst occurs every 30 seconds for a duration of 5 seconds,
        # but only for t_sim >= 30 s.
        # The burst index is computed as floor(t_sim/30). For example:
        #   For t in [30,35): burst_index = 1, so wind_dir_deg = original + 30.
        #   For t in [60,65): burst_index = 2, so wind_dir_deg = original + 60.
        if t_sim >= 30 and (t_sim % 30) < 5:
            burst_index = int(t_sim // 30)
            sim_params["wind_dir_deg"] = original_wind_dir + 30 * burst_index
        else:
            sim_params["wind_dir_deg"] = original_wind_dir

            
        # 2) Compute anchor force
        tau_anchor, anchor_mag = compute_anchor_force(ship_params, sim_params, x, y, psi)
        tau_wind = compute_wind_force_3dof(u, v, psi, ship_params, sim_params)

        # # 4) Engage MPC control based on tension force
        if anchor_mag > tension_threshold:
            # Set external parameters (e.g., wind) for do_mpc
            # mpc.set_uncertainty_values(F_wind_x=[Fx_wind_g], F_wind_y=[Fy_wind_g])
            # Get control input from MPC
            u_opt = mpc.make_step(state)
            tau_prop = np.array(u_opt).flatten()  # [tau_x, tau_y, tau_psi]
                
                # Update previous inputs
            tau_x_prev = tau_prop[0]
            tau_y_prev = tau_prop[1]
            tau_psi_prev = tau_prop[2]

        else:
            # No propulsion when tension is below the threshold
            tau_prop = np.array([0.0, 0.0, 0.0])

        # tau_prop = np.array([0.0, 0.0, 0.0])




        # to add relative velocities to RESULTS 
        cos_psi = np.cos(psi)
        sin_psi = np.sin(psi)
        # R^T(psi) = [[ cos(psi), sin(psi)],
        #             [-sin(psi), cos(psi)]]
        u_c_b =  Ucx_g*cos_psi + Ucy_g*sin_psi
        v_c_b = -Ucx_g*sin_psi + Ucy_g*cos_psi

        # 4) Relative velocities
        u_r = u - u_c_b
        v_r = v - v_c_b


        # 5) Sum external forces in the body frame
        tau_ext = tau_anchor + tau_wind + tau_prop

        tau_wind_total = np.sqrt(tau_wind[0]**2 + tau_wind[1]**2)

        # Compute bow coords in GLOBAL
        Lpp = ship_params["Lpp"]
        x_bow = x + 0.5 * Lpp * np.cos(psi)
        y_bow = y + 0.5 * Lpp * np.sin(psi)

        # Distance from anchor (0,0)
        rode_length = np.hypot(x_bow, y_bow)

        

        # RK4 integration
        k1, nu_dot = compute_dstate(ship_params, sim_params, state, tau_ext)
        k2, _ = compute_dstate(ship_params, sim_params, state + 0.5*dt*k1, tau_ext)
        k3, _ = compute_dstate(ship_params, sim_params, state + 0.5*dt*k2, tau_ext)
        k4 , _ = compute_dstate(ship_params, sim_params, state + dt*k3,     tau_ext)
        state = state + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)


        # 7) Store logs
        results.append({
            "t": t_sim,
            "x": x, "y": y, "psi": psi,
            "u": u, "v": v, "r": r,
            "u_r": u_r,
            "v_r": v_r,
            "rode_length": rode_length,

            "Fx_prop": tau_prop[0],
            "Fy_prop": tau_prop[1],
            "Mz_prop": tau_prop[2],

            "Fx_wind": tau_wind[0],
            "Fy_wind": tau_wind[1],
            "Mz_wind": tau_wind[2],
            "wind_mag": tau_wind_total,

            "wind_speed": sim_params.get("wind_speed", 0.0),
            "wind_dir_deg": sim_params["wind_dir_deg"],
            
            "Fx_total_ext": tau_ext[0],
            "Fy_total_ext": tau_ext[1],
            "Mz_total_ext": tau_ext[2],

            "Fx_anchor": tau_anchor[0],
            "Fy_anchor": tau_anchor[1],
            "Mz_anchor": tau_anchor[2],
            "anchor_mag": anchor_mag,  # log the anchor tension for debugging
        })

        # Advance time
        t_sim += dt
    
    # Restore the original wind direction in sim_params.
    sim_params["wind_dir_deg"] = original_wind_dir

    # Convert results to a Pandas DataFrame
    df = pd.DataFrame(results)
    return df

